In [5]:
from gensim.models.word2vec import Word2Vec
from gensim.models.fasttext import FastText
import gensim
import os,json
import multiprocessing
from collections import defaultdict
from tensorflow.python.client import device_lib
def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]
print(get_available_devices())
cores = multiprocessing.cpu_count()
print("['CPU cores:",cores,"']")
script_path = os.path.dirname(os.path.abspath('__file__'))
stopwords = set(
    ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves',
     'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their',
     'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was',
     'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the',
     'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against',
     'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in',
     'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why',
     'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only',
     'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'now'])
STOP=stopwords

In [6]:
def replacewith(input_str, pattern, replaceWith): 
    return input_str.replace(pattern, replaceWith)
def extract_data(data_file,polarity_map,max_len=80):
    data = open(data_file,'r').readlines()
    corpus=[]
    for i in range(0,len(data),3):
        temp={}
        if len(data[i].split("\n")[0])>=max_len:
            temp["sentence"] = data[i].split("\n")[0]
            temp["aspect_term"] = data[i+1].split("\n")[0]
            temp["polarity"] = polarity_map[data[i+2].split("\n")[0]]
            corpus.append(temp)
    return corpus 

In [7]:
polarity_map={"1":"positive","-1":"negative","0":"neutral"}
train_data=script_path+"/data/restaurant/rest_2014_train.txt"
test_data=script_path+"/data/restaurant/rest_2014_test.txt"
train= extract_data(train_data,polarity_map,max_len=0)
test = extract_data(test_data,polarity_map,max_len=0)
print("Number of training sentences: ",len(train))
print("Number of testing sentences: ",len(test))

Number of training sentences:  3699
Number of testing sentences:  1134


In [9]:
def load_sentences(data):
    sentences=[]
    for x in data:
        sent = replacewith(x["sentence"],"$T$",x["aspect_term"])
        sentences.append(sent.split())
    return sentences    

In [12]:
train_sents = load_sentences(train)
test_sents = load_sentences(test)
print("Number of training sentences: ",len(train_sents))
print("Number of testing sentences: ",len(test_sents))

Number of training sentences:  3699
Number of testing sentences:  1134


In [15]:
# train model
cbow_model = Word2Vec(train_sents,size=300,window=3,min_count=1,workers=cores)
skip_model = Word2Vec(train_sents,size=300,window=3,min_count=1,workers=cores,sg=1)

40


In [19]:
model = cbow_model

(1044830, 1488340)

In [ ]:
## To handle OOV words we use the concept used in Facebook Fasttext library of word embeddings
from gensim.models.wrappers import FastText


In [20]:
# summarize the loaded model
print(model)
# summarize vocabulary
#words = list(model.wv.vocab)
#print(words)
w1='dirty'
model.wv.most_similar(positive=w1)
# access vector for one word
print(model['sentence'])
# save model
model.save('model.bin')
# load model
new_model = Word2Vec.load('model.bin')
print(new_model)

Word2Vec(vocab=4435, size=300, alpha=0.025)


KeyError: "word 'dirty' not in vocabulary"